En este código se realiza lo siguiente:

01. Se importa la base de datos meteorológicos proporcionada por Insivumeh
02. Desechamos todas las variables excepto la precipitación
03. Rellenamos valores faltantes de precipitación por lo que indican la base de datos de ENACTS, cuyos datos son importados en un archivo en formato NetCDF de nombre precipitacion.nc
04. Se calcula el acumulado por estaciones para cada estación meteorológica (las estaciones son DEFM, MJJ, ASO)
05. Se calcula la media y los percentiles 1/3 y 2/3, así como la desviación estándar del conjunto de precipitación acumuladas de cada perído estacional para la Climatología (1991-2020). Este proceso se realiza para cada una de las estaciones meteorológicas.
06. Se exporta un archivo de Excel que se incluya el nombre de la estación, el período estacional, y la media y los percentiles 1/3 y 2/3 del conjunto de precipitaciones acumuladas en la Climatología.

# IMPORTACIÓN DE DATOS Y LIBRERÍAS

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
file_path = 'BaseDatos.xlsx'
Datos = pd.read_excel(file_path)

print(Datos)

# ELIMINACIÓN DE TODAS LAS VARIABLES EXCEPTO PRECIPITACION

In [ ]:
VarEliminar = ['Humedad_R', 'Tmin', 'Tmax', 'Tmed', 'Radiacion','Brillo_Solar','Evap_Tanque','Presion_atms', 'Piche', 'Vel_viento', 'Dir_viento']

Datos.drop(columns=VarEliminar, inplace=True)

print(Datos)

# AGREGAR DÍAS NO REGISTRADOS

EN PRIMER LUGAR, SE DEBE GENERAR UN REGISTRO DE LOS DÍAS QUE NO ESTÁN INGRESADOS, PARA CADA UNA DE LAS ESTACIONES

In [ ]:
# Crear un DataFrame que contenga todas las fechas posibles en el rango de fechas para cada estación
start_date = Datos['FECHA'].min()
end_date = Datos['FECHA'].max()
all_dates = pd.date_range(start=start_date, end=end_date, freq='D')

# Crear una lista para almacenar los resultados
missing_dates = []

# Iterar sobre cada estación
for estacion in Datos['ESTACIÓN'].unique():
    # Filtrar los datos por estación
    Datos_estacion = Datos[Datos['ESTACIÓN'] == estacion]
    
    # Obtener las fechas presentes en la base de datos para esta estación
    fechas_present = Datos_estacion['FECHA']
    
    # Obtener las fechas faltantes para esta estación
    fechas_missing = all_dates.difference(fechas_present)
    
    # Agregar las fechas faltantes a la lista de resultados
    for date in fechas_missing:
        missing_dates.append({'ESTACIÓN': estacion, 'FECHA': date})

# Convertir la lista de resultados a un DataFrame
missing_dates_Datos = pd.DataFrame(missing_dates)

print("Las fechas no ingresadas en la base de datos son: ")
print(missing_dates_Datos)

A CONTINUACIÓN, SE AGREGAN A LA BASE DE DATOS LOS DÍAS NO INGRESADOS; TODOS LOS VALORES DE ESTOS DÍAS APARECERAN VACÍOS

In [ ]:
# Añadir las columnas de datos meteorológicos con valores NaN
for col in Datos.columns:
    if col not in ['ESTACIÓN', 'FECHA']:
        missing_dates_Datos[col] = np.nan

# Combinar el DataFrame original con el DataFrame de fechas faltantes
Datos = pd.concat([Datos, missing_dates_Datos], ignore_index=True)

# Ordenar el DataFrame resultante por estación y fecha
Datos = Datos.sort_values(by=['ESTACIÓN', 'FECHA'])

print(Datos)

# AGRAGAR LAT, LON

Se agregan estos datos como columnas adicionales a la base de datos, ya que serán útiles al realizar el rellenado de datos faltantes

In [ ]:
# Importar la base de datos que contiene los nombres de las estaciones y su latitud, longitud y altura.
file_path = 'Datos_estaciones.xlsx'
Datos_estaciones = pd.read_excel(file_path)

# Fusionar la base de datos con los datos de las estaciones, en función del nombre de la estación
Datos = pd.merge(Datos, Datos_estaciones, on='ESTACIÓN', how='left')

# Eliminar las columnas que, para nuestro caso, son irrelevantes
VarEliminar = ['Altitud', 'Departamento', 'Municipio']
Datos.drop(columns=VarEliminar, inplace=True)

print(Datos)

In [ ]:
# Guardamos el avance en un dataframe
Preliminar1 = Datos

# RELLENO DE DATOS FALTANTES DE PRECIPITACIÓN

SE SUSTITUYEN LOS DATOS FALTANTES POR LOS QUE INDICA LA BASE DE DATOS DE ENACTS

In [ ]:
# Verificar cuántos valores son desconocidos
print("Valores vaciós de precipitación: " + str(Datos['Lluvia'].isna().sum()))

In [ ]:
import xarray as xr
import warnings

# Suprimir advertencias de tipo FutureWarning
warnings.filterwarnings('ignore', category=FutureWarning)

# Crear una copia del DataFrame original, la cual se irá rellenando
Datos_rellenados = Datos.copy()

# Mapeo de variable de la base de datos a las variable en el archivo NetCDF (esto se hace ya que el nombre de la variable
# en la base de datos meteorológicos puede no tener el mismo nombre que la variable en el archivo NetCDF)
var_map = {
    'Lluvia': 'rfe',
}

# Archivo NetCDF correspondiente
nc_files = {
    'rfe': 'precipitacion.nc',
}

# Cargar los archivos NetCDF en un diccionario
nc_data = {var: xr.open_dataset(file) for var, file in nc_files.items()}

# Iterar sobre las filas
for i, row in Datos.iterrows():
    # Extraer latitud, longitud y fecha para la fila actual
    lat = row['Latitud']
    lon = row['Longitud']
    fecha = row['FECHA']
    estacion = row['ESTACIÓN']
    
    # Iterar sobre las variables para la fila actual
    for db_var, nc_var in var_map.items():
        if pd.isna(row[db_var]):
            # Extraer los datos de la variable correspondiente del archivo NetCDF
            valor = nc_data[nc_var][nc_var].sel(
                Y=lat, X=lon, T=fecha, method='nearest'
            ).values
            
            # Reemplazar el valor en el DataFrame
            Datos_rellenados.at[i, db_var] = valor
            
    print("Se han rellenado las variables en la estación " + "'" + estacion + "'" + " en la fecha " + str(fecha))

print(Datos_rellenados)

In [ ]:
# Actualizar el DataFrame original con los datos rellenados
Datos = Datos_rellenados

In [ ]:
# Verificar que no quede ningún valor vacío
print("Valores vaciós de precipitación: " + str(Datos['Lluvia'].isna().sum()))

In [ ]:
# Finalmente, guardamos el avance en un dataframe
Preliminar2 = Datos

# GENERACIÓN DE DATOS DE LA CLIMATOLOGÍA POR PERÍODO ESTACIONAL PARA CADA UNA DE LAS ESTACIONES METEOROLÓGICAS

In [ ]:
# Definir los períodos estacionales
def obtener_periodo_estacional(mes):
    if mes in [11, 12, 1]:
        return 'DEFM'  # Diciembre, Enero, Febrero, Marzo
    elif mes in [5, 6, 7]:
        return 'MJJ'  # Mayo, Junio, Julio
    elif mes in [8, 9, 10]:
        return 'ASO'  # Agosto, Septiembre, Octubre

In [ ]:
# Agregar una columna de "Período estacional"
Datos['Periodo_estacional'] = Datos['FECHA'].dt.month.apply(obtener_periodo_estacional)

# Agregar una columna para el año
Datos['Año'] = Datos['FECHA'].dt.year

print(Datos)

In [ ]:
# Filtrar los datos para la climatología que abarca de diciembre de 1990 hasta octubre de 2020
Datos_climatologia = Datos[
    ((Datos['Año'] == 1990) & (Datos['FECHA'].dt.month == 12)) |  # Desde diciembre 1990
    ((Datos['Año'] > 1990) & (Datos['Año'] < 2020)) |  # Años completos entre 1992 y 2020
    ((Datos['Año'] == 2020) & (Datos['FECHA'].dt.month <= 10))  # Hasta octubre 2020
]

print(Datos_climatologia)

In [ ]:
# Agrupar los datos por estación, año y período estacional y calcular el acumulado de precipitación
acumulados = Datos_climatologia.groupby(['ESTACIÓN', 'Año', 'Periodo_estacional'])['Lluvia'].sum().reset_index()

print(acumulados)

In [ ]:
# Ahora, calcular la media, percentil 1/3 y 2/3 de los acumulados de cada estación y período estacional para los años de climatología
estadisticos = acumulados.groupby(['ESTACIÓN', 'Periodo_estacional']).agg(
    Media=('Lluvia', 'mean'),
    Percentil_1_3=('Lluvia', lambda x: np.percentile(x, 33.333333333)),
    Percentil_2_3=('Lluvia', lambda x: np.percentile(x, 66.666666666))
).reset_index()

print(estadisticos)

In [ ]:
# Exportar los resultados a un archivo Excel
estadisticos.to_excel('Climatologia_por_Periodos_Estacionales.xlsx', index=False)